In [ ]:
import pandas as pd

In [ ]:
# import path
path = r"C:\Users\calli\Downloads\PROP-database-new\da_aggiungere\fnext\100\f\3.htm"

# download path
download_path = r"C:\Users\calli\Downloads\PROP-database-new\da_aggiungere\fnext\100\f"

In [ ]:
def html_to_excel(html_file, excel_file):
    try:
        # Read HTML tables from the file
        tables = pd.read_html(html_file)  # Returns a list of DataFrames
        if not tables:
            print("No tables found in the HTML file.")
            return
        
        # Save each table to a separate sheet in Excel
        with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
            for idx, table in enumerate(tables, start=1):
                sheet_name = f"Sheet{idx}"
                table.to_excel(writer, sheet_name=sheet_name, index=False)
        
        print(f"Conversion successful! Saved to '{excel_file}'")
    
    except FileNotFoundError:
        print(f"Error: File '{html_file}' not found.")
    except ValueError as e:
        print(f"Error reading HTML: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")

# Example usage
html_to_excel("input.html", "output.xlsx")